In [1]:
import cv2
import numpy as np
import os

# Define the color enhancement function with CLAHE for color correction
def color_enhancement(image, clipLimit=1.0, tileGridSize=(12, 12)):
    # Convert BGR image to LAB color space
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    l, a, b = cv2.split(lab)  # Split into L, A, and B channels
    # Create CLAHE object and apply to the L channel
    clahe = cv2.createCLAHE(clipLimit=clipLimit, tileGridSize=tileGridSize)
    l = clahe.apply(l)
    # Merge enhanced L channel with original A and B channels
    lab = cv2.merge((l, a, b))
    # Convert back to BGR color space
    enhanced_image = cv2.cvtColor(lab, cv2.COLOR_LAB2BGR)
    return enhanced_image

# Define the function to check for blurriness
def is_blurry(image, threshold=100):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    variance_of_laplacian = cv2.Laplacian(gray, cv2.CV_64F).var()
    return variance_of_laplacian < threshold

# Define the function to resize the image
def resize_image(image, size=(224, 224)):
    resized_image = cv2.resize(image, size, interpolation=cv2.INTER_AREA)
    return resized_image

# Define the function for Min-Max normalization
def min_max_normalization(image):
    normalized_image = image.astype(np.float32) / 255.0  # Normalize by dividing by 255
    return normalized_image

# Define the function to select and pad ROI
def select_and_pad_roi(image, roi_size=(224, 100)):
    height, width, _ = image.shape
    start_y = max(0, (height - roi_size[0]) // 2)
    start_x = max(0, (width - roi_size[1]) // 2)
    end_y = start_y + roi_size[0]
    end_x = start_x + roi_size[1]
    roi = image[start_y:end_y, start_x:end_x]

    # Pad ROI to 224x224
    padded_roi = cv2.copyMakeBorder(
        roi,
        top=0,
        bottom=0,
        left=(224 - roi_size[1]) // 2,
        right=(224 - roi_size[1]) // 2,
        borderType=cv2.BORDER_CONSTANT,
        value=[0, 0, 0]  # Black padding
    )
    return padded_roi

# Define the function to process and save the images
def process_and_save_images(input_folder, output_folder):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    for filename in os.listdir(input_folder):
        image_path = os.path.join(input_folder, filename)
        image = cv2.imread(image_path)
        
        if image is None:
            print(f"Error loading image: {image_path}")
            continue
        
        if is_blurry(image):
            print(f"The image {filename} is blurry. Skipping.")
            continue
        
        # Resize the image to 160x160
        resized_image = resize_image(image)
        
        # Enhance the resized image colors
        enhanced_image = color_enhancement(resized_image)
        
        # Normalize the enhanced image
        normalized_image = min_max_normalization(enhanced_image)
        
        # Select and pad ROI
        padded_roi = select_and_pad_roi(normalized_image, roi_size=(224, 100))
        
        # Convert normalized image back to uint8 for saving
        normalized_image_display = (padded_roi * 255).astype(np.uint8)
        
        # Save the processed image
        output_path = os.path.join(output_folder, filename)
        cv2.imwrite(output_path, normalized_image_display)
        print(f"Processed and saved: {output_path}")

# Define paths
input_folder = r'C:\CINNAMON\test_images'  # Replace with your dataset path
output_folder = r'C:\CINNAMON\output_folder'  # Replace with your output folder path

# Process and save images
process_and_save_images(input_folder, output_folder)


Processed and saved: C:\CINNAMON\output_folder\20230928_112847.jpg
The image 20230928_113415_0_1250.jpeg is blurry. Skipping.
Processed and saved: C:\CINNAMON\output_folder\20230928_113415_0_3913.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113415_0_5498.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113415_0_5698.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113415_0_8369.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113416_0_1317.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113416_0_2302.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113416_0_4857.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113416_0_7203.jpeg
Processed and saved: C:\CINNAMON\output_folder\20230928_113416_0_7326.jpeg
The image 20230928_113426_0_1368.jpeg is blurry. Skipping.
The image 20230928_113426_0_3973.jpeg is blurry. Skipping.
Processed and saved: C:\CINNAMON\output_folder\20230928_113426_0_5204.jpeg
The im